In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

batch_size = 128
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, n_classes])

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool2d(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [7]:
def convolutional_neural_network(x):#, keep_rate):
    conv1_depth = 30
    conv2_depth = 60
    weights = {
        # 5 x 5 convolution, 1 input image, 32 outputs
        # we assign 32 neurons per 5x5 window (we create a mini-NN for the given window)
        'W_conv1': tf.Variable(tf.random_normal([5, 5, 1, conv1_depth])),
        # 5x5 conv, 32 inputs, 64 outputs 
        'W_conv2': tf.Variable(tf.random_normal([5, 5, conv1_depth, conv2_depth])),
        # fully connected, 7*7*64 inputs, 1024 outputs
        'W_fc': tf.Variable(tf.random_normal([7*7*conv2_depth, 1024])),
        # 1024 inputs, 10 outputs (class prediction)
        'out': tf.Variable(tf.random_normal([1024, n_classes]))
    }

    biases = {
        'b_conv1': tf.Variable(tf.random_normal([conv1_depth])),
        'b_conv2': tf.Variable(tf.random_normal([conv2_depth])),
        'b_fc': tf.Variable(tf.random_normal([1024])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    # Reshape input to a 4D tensor 
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    # Convolution Layer, using our function
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1)
    # Convolution Layer
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2)
    
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer
    fc = tf.reshape(conv2, [-1, 7*7*conv2_depth])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc']) + biases['b_fc'])
    
    output = tf.matmul(fc, weights['out']) + biases['out']
    return output

def train_neural_network(x):
    # initialize prediction flow using dummy x
    prediction = convolutional_neural_network(x)
    # initialize cost calculation flow using dummy y
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    # initialize optimizer flow using cost
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        # need to initialize all vars defined by tf.Variable()
        sess.run(tf.initialize_all_variables())

        # each epoch is a run through the full training data
        for epoch in range(hm_epochs):
            epoch_loss = 0
            # separate data out into batches
            # for each batch, optimize on the cost function
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
            
        # define correct indicator function
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        # define accuracy function
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        # print final accuracy on test data
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [8]:
prediction = train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 10 loss: 1298060.34874
Epoch 1 completed out of 10 loss: 216223.729683
Epoch 2 completed out of 10 loss: 127699.624783
Epoch 3 completed out of 10 loss: 85444.8962097
Epoch 4 completed out of 10 loss: 57426.2317467
Epoch 5 completed out of 10 loss: 44069.2269936
Epoch 6 completed out of 10 loss: 30785.6979713
Epoch 7 completed out of 10 loss: 25476.9337015
Epoch 8 completed out of 10 loss: 23594.827346
Epoch 9 completed out of 10 loss: 18496.5780658
Accuracy: 0.9765
